# Environment preparation


## connect to Google drive


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !pip install sidetable

In [ ]:
# !pip install ydata-profiling

In [ ]:
# !jupyter nbextension enable --py widgetsnbextension

## import library


In [ ]:
import os

import numpy as np
import pandas as pd
import sidetable
from ydata_profiling import ProfileReport

# Load dataset


In [ ]:
file_path = r"data\MolochTHbackup.xlsx"
df = pd.read_excel(file_path, sheet_name=6, header=1, usecols=range(1, 8))
df.drop("GuildPic", axis=1, inplace=True)
df.rename({"Season\nRank": "Rank", "Total Trophies": "Trophies"}, axis=1, inplace=True)
df.dropna(subset="Trophies", inplace=True)
df["Trophies"] = df.Trophies.astype(int)
df = df[df["Season"] != 10]
df

d:\repos\CRK_guild_boss\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Date,Season,Round,Rank,GuildName,Trophies
62,2024-02-27,1,1-1,1,Spearmint,10513190
63,2024-02-27,1,1-1,2,Vivid,10510940
64,2024-02-27,1,1-1,3,Nobless,10251975
65,2024-02-27,1,1-1,4,LAODA,10011795
66,2024-02-27,1,1-1,5,미라클,9930645
...,...,...,...,...,...,...
262,2024-04-30,3,3-2,16,Nobility,18909650
263,2024-04-30,3,3-2,17,S7AR,18647100
264,2024-04-30,3,3-2,18,DEKWAT,18422920
265,2024-04-30,3,3-2,19,MyTH,18315910


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 205 entries, 62 to 266
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       205 non-null    datetime64[ns]
 1   Season     205 non-null    int64         
 2   Round      205 non-null    object        
 3   Rank       205 non-null    int64         
 4   GuildName  205 non-null    object        
 5   Trophies   205 non-null    int32         
dtypes: datetime64[ns](1), int32(1), int64(2), object(2)
memory usage: 10.4+ KB


# Add Weekly Earned Trophies column


In [ ]:
# Sort values by Guild Name and Round
data = df.sort_values(by=["GuildName", "Round"])

# Look up the Trophies from the previous round
data["PrevRoundTrophies"] = data["Trophies"].shift(periods=1).fillna(0).astype(int)

# Replace the previous round values in rows with minimum Trophies of each season with zero
for row in data.itertuples():
    guild_name = row.GuildName
    season = row.Season
    if (
        row.Trophies
        == data[(data.GuildName == guild_name) & (data.Season == season)].Trophies.min()
    ):
        data.loc[row.Index, "PrevRoundTrophies"] = 0

# Calc the weekly earned
data["Earned"] = data["Trophies"] - data["PrevRoundTrophies"]

# drop look up column and rename Trophies
data.drop(columns="PrevRoundTrophies", inplace=True)
data.rename(columns={"Trophies": "SeasonTotal", "Earned": "Trophies"}, inplace=True)
data = data[["Date", "Season", "Round", "Rank", "GuildName", "Trophies", "SeasonTotal"]]
data.sort_values(by=["Round", "Rank"], inplace=True)
data.reset_index(drop=True, inplace=True)
data

,Date,Season,Round,Rank,GuildName,Trophies,SeasonTotal
0,2024-02-27,1,1-1,1,Spearmint,10513190,10513190
1,2024-02-27,1,1-1,2,Vivid,10510940,10510940
2,2024-02-27,1,1-1,3,Nobless,10251975,10251975
3,2024-02-27,1,1-1,4,LAODA,10011795,10011795
4,2024-02-27,1,1-1,5,미라클,9930645,9930645
...,...,...,...,...,...,...,...
200,2024-04-30,3,3-2,16,Nobility,9456020,18909650
201,2024-04-30,3,3-2,17,S7AR,9286535,18647100
202,2024-04-30,3,3-2,18,DEKWAT,9213865,18422920
203,2024-04-30,3,3-2,19,MyTH,9118120,18315910


# Validate calc values


In [ ]:
# max total must equal to sum of earned trophies for each season
compare_total = data.groupby(["GuildName", "Season"], as_index=False).agg(
    maxTotal=("SeasonTotal", "max"), sumEarned=("Trophies", "sum")
)

display(compare_total)


# if all rows are true, sum of boonlean values must equal to number of rows
val_result = (compare_total.maxTotal == compare_total.sumEarned).sum() == len(
    compare_total
)

print(f"maxTotal = sumEarned : {val_result}")

,GuildName,Season,maxTotal,sumEarned
0,Crown,1,39348520,39348520
1,Crown,2,40924725,40924725
2,Crown,3,21025605,21025605
3,DEKWAT,1,34182330,34182330
4,DEKWAT,2,36453935,36453935
...,...,...,...,...
57,재난경보,2,37689285,37689285
58,재난경보,3,19448515,19448515
59,쿠벤저스,1,33869870,33869870
60,쿠벤저스,2,35692120,35692120


maxTotal = sumEarned : True


In [ ]:
max_date = str(data["Date"].max()).split(" ")[0]
max_date

'2024-04-30'

In [ ]:
max_round = data["Round"].max()
max_round

'3-2'

# Save df in .csv format


In [ ]:
save_path = r"data\CRK_guild_boss.csv"
# save_path

data.to_csv(save_path, index=False)

# EDA


In [ ]:
report = ProfileReport(data, title="EDA report")

## displaying the report as a set of widgets


In [ ]:
# report.to_widgets()

In [ ]:
report.to_notebook_iframe()

## generate a HTML report file


In [24]:
save_path = r"report/CRK_guild_boss_EDA_report.html"
# save_path

report.to_file(save_path)

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 200.07it/s]


# Report table


In [16]:
import plotly.graph_objects as go
import numpy as np

title = "Main Source for News"
labels = ["Television", "Newspaper", "Internet", "Radio"]
colors = ["rgb(67,67,67)", "rgb(115,115,115)", "rgb(49,130,189)", "rgb(189,189,189)"]

mode_size = [8, 8, 12, 8]
line_size = [2, 2, 4, 2]

x_data = np.vstack((np.arange(2001, 2014),) * 4)

y_data = np.array(
    [
        [74, 82, 80, 74, 73, 72, 74, 70, 70, 66, 66, 69],
        [45, 42, 50, 46, 36, 36, 34, 35, 32, 31, 31, 28],
        [13, 14, 20, 24, 20, 24, 24, 40, 35, 41, 43, 50],
        [18, 21, 18, 21, 16, 14, 13, 18, 17, 16, 19, 23],
    ]
)

fig = go.Figure()

for i in range(0, 4):
    fig.add_trace(
        go.Scatter(
            x=x_data[i],
            y=y_data[i],
            mode="lines",
            name=labels[i],
            line=dict(color=colors[i], width=line_size[i]),
            connectgaps=True,
        )
    )

    # endpoints
    fig.add_trace(
        go.Scatter(
            x=[x_data[i][0], x_data[i][-1]],
            y=[y_data[i][0], y_data[i][-1]],
            mode="markers",
            marker=dict(color=colors[i], size=mode_size[i]),
        )
    )

fig.update_layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor="rgb(204, 204, 204)",
        linewidth=2,
        ticks="outside",
        tickfont=dict(
            family="Arial",
            size=12,
            color="rgb(82, 82, 82)",
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=False,
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=False,
    plot_bgcolor="white",
)

annotations = []

# Adding labels
for y_trace, label, color in zip(y_data, labels, colors):
    # labeling the left_side of the plot
    annotations.append(
        dict(
            xref="paper",
            x=0.05,
            y=y_trace[0],
            xanchor="right",
            yanchor="middle",
            text=label + " {}%".format(y_trace[0]),
            font=dict(family="Arial", size=16),
            showarrow=False,
        )
    )
    # labeling the right_side of the plot
    annotations.append(
        dict(
            xref="paper",
            x=0.95,
            y=y_trace[11],
            xanchor="left",
            yanchor="middle",
            text="{}%".format(y_trace[11]),
            font=dict(family="Arial", size=16),
            showarrow=False,
        )
    )
# Title
annotations.append(
    dict(
        xref="paper",
        yref="paper",
        x=0.0,
        y=1.05,
        xanchor="left",
        yanchor="bottom",
        text="Main Source for News",
        font=dict(family="Arial", size=30, color="rgb(37,37,37)"),
        showarrow=False,
    )
)
# Source
annotations.append(
    dict(
        xref="paper",
        yref="paper",
        x=0.5,
        y=-0.1,
        xanchor="center",
        yanchor="top",
        text="Source: PewResearch Center & " + "Storytelling with data",
        font=dict(family="Arial", size=12, color="rgb(150,150,150)"),
        showarrow=False,
    )
)

fig.update_layout(annotations=annotations)

fig.show()

In [21]:
rep_table = data.pivot_table(
    index="GuildName",
    columns=["Season", "Round"],
    values=["Rank", "Trophies"],
    aggfunc="max",
)


# rep_table.columns[0]


rep_table.sort_values(by=rep_table.columns[0], inplace=True)


rep_table.style.format(
    precision=0,
    thousands=",",
    decimal=".",
)  #   .format_index(str.upper, axis=1) \


#   .relabel_index(["row 1", "row 2"], axis=0)

In [28]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 205 entries, 62 to 266
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         205 non-null    datetime64[ns]
 1   Season       205 non-null    int64         
 2   Round        205 non-null    object        
 3   Rank         205 non-null    int64         
 4   GuildName    205 non-null    object        
 5   Trophies     205 non-null    int64         
 6   SeasonTotal  205 non-null    int64         
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 20.9+ KB


In [ ]:
data.nunique()

Date            10
Season           3
Round           10
Rank            24
GuildName       21
Trophies       205
SeasonTotal    204
dtype: int64

In [ ]:
for col in data.columns:
    if data[col].nunique() < 30:
        print(f"- {col}")
        display(data.stb.freq([col], style=True))
        print("\n")

- Date


,Date,count,percent,cumulative_count,cumulative_percent
0,2024-03-26 00:00:00,21,10.24%,21,10.24%
1,2024-03-19 00:00:00,21,10.24%,42,20.49%
2,2024-03-12 00:00:00,21,10.24%,63,30.73%
3,2024-03-05 00:00:00,21,10.24%,84,40.98%
4,2024-02-27 00:00:00,21,10.24%,105,51.22%
5,2024-04-30 00:00:00,20,9.76%,125,60.98%
6,2024-04-23 00:00:00,20,9.76%,145,70.73%
7,2024-04-16 00:00:00,20,9.76%,165,80.49%
8,2024-04-09 00:00:00,20,9.76%,185,90.24%
9,2024-04-02 00:00:00,20,9.76%,205,100.00%




- Season


,Season,count,percent,cumulative_count,cumulative_percent
0,1,84,40.98%,84,40.98%
1,2,81,39.51%,165,80.49%
2,3,40,19.51%,205,100.00%




- Round


,Round,count,percent,cumulative_count,cumulative_percent
0,2-1,21,10.24%,21,10.24%
1,1-4,21,10.24%,42,20.49%
2,1-3,21,10.24%,63,30.73%
3,1-2,21,10.24%,84,40.98%
4,1-1,21,10.24%,105,51.22%
5,3-2,20,9.76%,125,60.98%
6,3-1,20,9.76%,145,70.73%
7,2-4,20,9.76%,165,80.49%
8,2-3,20,9.76%,185,90.24%
9,2-2,20,9.76%,205,100.00%




- Rank


,Rank,count,percent,cumulative_count,cumulative_percent
0,20,10,4.88%,10,4.88%
1,19,10,4.88%,20,9.76%
2,18,10,4.88%,30,14.63%
3,17,10,4.88%,40,19.51%
4,16,10,4.88%,50,24.39%
5,15,10,4.88%,60,29.27%
6,14,10,4.88%,70,34.15%
7,13,10,4.88%,80,39.02%
8,12,10,4.88%,90,43.90%
9,11,10,4.88%,100,48.78%




- GuildName


,GuildName,count,percent,cumulative_count,cumulative_percent
0,쿠벤저스,10,4.88%,10,4.88%
1,재난경보,10,4.88%,20,9.76%
2,이지스,10,4.88%,30,14.63%
3,열매,10,4.88%,40,19.51%
4,사혼,10,4.88%,50,24.39%
5,미라클,10,4.88%,60,29.27%
6,무야호,10,4.88%,70,34.15%
7,님,10,4.88%,80,39.02%
8,Vivid,10,4.88%,90,43.90%
9,Spearmint,10,4.88%,100,48.78%


# Create report table by season


In [ ]:
for season in data.Season.unique().tolist():
    print(f"\n- Season {season}")
    earned_table = data[data["Season"] == season].pivot_table(
        index=["GuildName"],
        columns="Round",
        values=["Rank", "Trophies"],
        aggfunc="max",
    )
    earned_table.sort_values(
        by=earned_table.columns[0], ignore_index=False, inplace=True
    )
    # earned_table.columns = earned_table.columns.sortlevel()#.swaplevel(0, 1)
    print(earned_table.columns)

    display(
        earned_table.style.format(
            precision=0,
            thousands=",",
            decimal=".",
        )
    )


- Season 1
MultiIndex([(    'Rank', '1-1'),
            (    'Rank', '1-2'),
            (    'Rank', '1-3'),
            (    'Rank', '1-4'),
            ('Trophies', '1-1'),
            ('Trophies', '1-2'),
            ('Trophies', '1-3'),
            ('Trophies', '1-4')],
           names=[None, 'Round'])


Rank              Trophies                              
Round       1-1 1-2 1-3 1-4       1-1       1-2       1-3       1-4
GuildName                                                          
Spearmint     1   1   1   1  10513190  10692545  10770870  10779410
Vivid         2   2   2   2  10510940  10598695  10690270  10702360
Nobless       3   3   3   3  10251975  10259645  10424260  10349095
LAODA         4   4   4   4  10011795  10091870  10103600  10174935
미라클           5   5   5   5   9930645   9930695  10096910  10263565
님             6   6   6   6   9688065  10173275   9678260  10005140
오로라           7   8   8   8   9677210   9682900   9772030   9931935
Crown         8   7   7   7   9591650   9836010   9835780  10085080
무야호           9  10  10   9   9525015   9613965   9678470   9759470
열매           10   9   9  10   9516665   9683165   9685015   9686065
MolochTH     11  11  11  11   9356625   9530190   9763490   9841245
MyTH         12  12  13  13   9351425   9522435   9360200   9595775
이지스          13  13  12  12   9286600   9367650   9614385   9936100
재난경보         14  14  15  15   9201780   9198000   8968255   9049535
Peppermint   15  15  14  14   8974230   9284540   9436210   9678895
사혼           16  16  16  16   8530020   8826920   8968345   9132590
Nigiri       17  17  17  17   8445275   8671380   8671310   8894760
S7AR         18  18  18  18   8439280   8519500   8661350   8891645
DEKWAT       19  19  19  19   8241380   8447715   8598275   8894960
쿠벤저스         20  20  20  20   8157340   8287605   8681615   8743310
Nobility     22  23  24  22   8022685   8091280   8088650   8754070


- Season 2
MultiIndex([(    'Rank', '2-1'),
            (    'Rank', '2-2'),
            (    'Rank', '2-3'),
            (    'Rank', '2-4'),
            ('Trophies', '2-1'),
            ('Trophies', '2-2'),
            ('Trophies', '2-3'),
            ('Trophies', '2-4')],
           names=[None, 'Round'])


Rank                      Trophies                          \
Round        2-1   2-2   2-3   2-4         2-1         2-2         2-3   
GuildName                                                                
Spearmint    1.0   1.0   1.0   1.0  10949190.0  11036070.0  11037250.0   
Vivid        2.0   2.0   2.0   2.0  10866400.0  10945065.0  11028425.0   
Nobless      3.0   3.0   3.0   3.0  10517570.0  10532845.0  10762930.0   
미라클          4.0   4.0   5.0   5.0  10343795.0  10352155.0  10346675.0   
님            5.0   5.0   6.0   6.0  10262110.0  10350005.0  10351645.0   
LAODA        6.0   6.0   4.0   4.0  10260975.0  10340180.0  10520225.0   
Crown        7.0   7.0   8.0   8.0  10165695.0  10165775.0  10084940.0   
이지스          8.0   8.0   7.0   7.0  10018860.0  10186940.0  10361535.0   
오로라          9.0   NaN   NaN   NaN  10007975.0         NaN         NaN   
열매          10.0  10.0  10.0  10.0   9936820.0   9934520.0  10108120.0   
MolochTH    11.0   9.0   9.0   9.0   9931170.0  10094570.0  10262850.0   
무야호         12.0  11.0  11.0  11.0   9849030.0   9855735.0   9931695.0   
Peppermint  13.0  12.0  12.0  12.0   9762440.0   9845765.0   9673590.0   
MyTH        14.0  14.0  15.0  15.0   9595795.0   9047905.0   9354565.0   
사혼          15.0  13.0  13.0  13.0   9285030.0   9436450.0   9523570.0   
재난경보        16.0  15.0  14.0  14.0   9203435.0   9436410.0   9520260.0   
S7AR        17.0  16.0  16.0  16.0   9047070.0   9200020.0   9200510.0   
Nigiri      18.0  18.0  19.0  18.0   8984405.0   8901670.0   8743210.0   
DEKWAT      19.0  17.0  17.0  17.0   8980530.0   9130900.0   9205955.0   
쿠벤저스        20.0  20.0  20.0  20.0   8901790.0   8839530.0   8822070.0   
Nobility    21.0  19.0  18.0  19.0   8833320.0   8991395.0   8917135.0   

                        
Round              2-4  
GuildName               
Spearmint   11044970.0  
Vivid       11128220.0  
Nobless     10767570.0  
미라클         10433690.0  
님           10426780.0  
LAODA       10600065.0  
Crown       10508315.0  
이지스         10433950.0  
오로라                NaN  
열매          10188930.0  
MolochTH    10256305.0  
무야호         10091360.0  
Peppermint   9756810.0  
MyTH         9047945.0  
사혼           9598815.0  
재난경보         9529180.0  
S7AR         9208600.0  
Nigiri       9360090.0  
DEKWAT       9136550.0  
쿠벤저스         9128730.0  
Nobility     9142300.0


- Season 3
MultiIndex([(    'Rank', '3-1'),
            (    'Rank', '3-2'),
            ('Trophies', '3-1'),
            ('Trophies', '3-2')],
           names=[None, 'Round'])


Rank      Trophies          
Round       3-1 3-2       3-1       3-2
GuildName                              
Spearmint     1   1  11216060  11211090
Vivid         2   2  11041545  11115440
Nobless       3   3  10792615  10781320
님             4   4  10621980  10683770
LAODA         5   5  10609375  10691550
이지스           6   6  10525165  10614170
미라클           7   7  10510435  10599700
MolochTH      8   9  10430590  10520085
Crown         9   8  10430030  10595575
열매           10  10  10265120  10350345
무야호          11  11  10101510  10258140
Peppermint   12  12   9929095  10084960
재난경보         13  13   9841595   9606920
사혼           14  14   9682255   9763190
Nobility     15  16   9453630   9456020
Nigiri       16  15   9442580   9525970
S7AR         17  17   9360565   9286535
DEKWAT       18  18   9209055   9213865
MyTH         19  19   9197790   9118120
쿠벤저스         20  20   9055195   9055055

---


# Load data from sheet name 'WeeklyEarned'


In [1]:
file_path = r"data\MolochTHbackup.xlsx"
df = pd.read_excel(file_path, sheet_name=7, header=0)
df.drop("EarnCalc", axis=1, inplace=True)
df

NameError: name 'pd' is not defined

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Date            205 non-null    datetime64[ns]
 1   Season          205 non-null    int64         
 2   Round           205 non-null    object        
 3   GuildName       205 non-null    object        
 4   RankTotal       205 non-null    int64         
 5   TotalTrophies   205 non-null    int64         
 6   EarnedTrophies  205 non-null    int64         
dtypes: datetime64[ns](1), int64(4), object(2)
memory usage: 11.3+ KB


In [47]:
df.columns

Index(['Date', 'Season', 'Round', 'GuildName', 'RankTotal', 'TotalTrophies',
       'EarnedTrophies'],
      dtype='object')

In [48]:
# max total must equal to sum of earned trophies for each season
compare_total = df.groupby(["GuildName", "Season"], as_index=False).agg(
    maxTotal=("TotalTrophies", "max"), sumEarned=("EarnedTrophies", "sum")
)

display(compare_total)


# if all rows are true, sum of boonlean values must equal to number of rows
val_result = (compare_total.maxTotal == compare_total.sumEarned).sum() == len(
    compare_total
)

print(f"maxTotal = sumEarned : {val_result}")

,GuildName,Season,maxTotal,sumEarned
0,Crown,1,39348520,39348520
1,Crown,2,40924725,40924725
2,Crown,3,21025605,21025605
3,DEKWAT,1,34182330,34182330
4,DEKWAT,2,36453935,36453935
...,...,...,...,...
57,재난경보,2,37689285,37689285
58,재난경보,3,19448515,19448515
59,쿠벤저스,1,33869870,33869870
60,쿠벤저스,2,35692120,35692120


maxTotal = sumEarned : True


In [50]:
# Pivot the data to prepare for animation
rank_changes = df.pivot_table(
    index="GuildName", columns="Round", values="RankTotal", aggfunc="max"
)
rank_changes.sort_values(by=rank_changes.columns[0], inplace=True)
rank_changes = rank_changes.applymap(lambda x: x * -1)  # type: ignore
rank_changes  # .reset_index()

Round,1-1,1-2,1-3,1-4,2-1,2-2,2-3,2-4,3-1,3-2
GuildName,,,,,,,,,,
Spearmint,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
Vivid,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0,-2.0
Nobless,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0,-3.0
LAODA,-4.0,-4.0,-4.0,-4.0,-6.0,-6.0,-4.0,-4.0,-5.0,-5.0
미라클,-5.0,-5.0,-5.0,-5.0,-4.0,-4.0,-5.0,-5.0,-7.0,-7.0
님,-6.0,-6.0,-6.0,-6.0,-5.0,-5.0,-6.0,-6.0,-4.0,-4.0
오로라,-7.0,-8.0,-8.0,-8.0,-9.0,NaN,NaN,NaN,NaN,NaN
Crown,-8.0,-7.0,-7.0,-7.0,-7.0,-7.0,-8.0,-8.0,-9.0,-8.0
무야호,-9.0,-10.0,-10.0,-9.0,-12.0,-11.0,-11.0,-11.0,-11.0,-11.0


In [49]:
rank_changes.columns

Index(['1-1', '1-2', '1-3', '1-4', '2-1', '2-2', '2-3', '2-4', '3-1', '3-2'], dtype='object', name='Round')

In [56]:
import plotly.express as px

In [58]:
top10_recent = df[(df.RankTotal <= 10) & (df.Season == 3)].GuildName.unique()

In [64]:
fig = px.line(
    df[df.GuildName.isin(top10_recent)],

    x="Round",

    y="EarnedTrophies",

    color="GuildName",

    markers=True,

    hover_data={
        "GuildName": False,
        "Date": "|%B %d, %Y",
        "RankTotal": ":,.0f",
        "EarnedTrophies": ":,.0f",
    },

    hover_name="GuildName",

    title="Guild Boss Trophies Over Time",

    labels={"RankTotal": "Rank", "Round": "Season", "EarnedTrophies": "Trophies"},

    template="simple_white",

    height=800,

)


fig.update_yaxes(showgrid=True)


fig.show()

In [66]:
fig = px.line(
    df,
    x="Round",
    y="RankTotal",

    color="GuildName",

    text="RankTotal",

    markers=True,
    line_shape="linear",

    labels={"GuildName": "Guild", "RankTotal": "Rank", "Round": "Season"},

    title="Guild Boss Ranking Over Time",

    template="simple_white",

    hover_name="GuildName",

    height=800,
    width=1000,

)


fig.update_yaxes(
    title="Rank", ticklabelstep=1, range=[25, 0], showgrid=False, visible=False

)


fig.update_xaxes(title="Season")

fig.update_layout(uniformtext_minsize=8)

fig.show()

In [ ]:
# Create the animated bar chart using Plotly Express
fig = px.bar(
    rank_changes,
    x="Rank",
    y="GuildName",
    orientation="h",
    animation_frame="Round",
    animation_group="GuildName",
    color="Rank",
    range_x=[-10, 1],  # Adjust range for inverted rank
    color_continuous_scale=px.colors.sequential.Plasma,
    title="Rank Changes for Top 10 Tourist Destinations (2019-2023)",
)

# # Customize the layout
# fig.update_layout(
#     xaxis_title="Inverted Rank (Higher is Better)",
#     yaxis_title="Country",
#     transition_duration=500,  # Set animation transition duration
# )

# # Show the animated chart
# fig.show()